In [7]:
import pandas as pd
from sqlalchemy import create_engine, text

In [8]:
url = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/sysarmy_survey_2020_processed.csv'
df = pd.read_csv(url)

In [9]:
df.head()

,profile_gender,profile_age,work_country,work_province,profile_years_experience,work_years_in_company,work_years_in_current_position,work_people_in_charge_of,profile_studies_level,profile_studies_level_state,...,work_has_violence_situations,profile_sexual_orientation,profile_has_disabilities,profile_has_disabilities_hiring_difficulties,company_employee_number,company_main_activity,company_recommended,company_diversity_policies,company_extra_benefits,company_best_companies_city
0,Mujer,26,Argentina,Ciudad Autónoma de Buenos Aires,3.0,3.0,3.0,0,Universitario,En curso,...,En mi trabajo actual,Homosexual,NaN,NaN,501-1000,Servicios / Consultoría de Software / Digital,7,2,"Capacitaciones y/o cursos, Comidas pagas / sub...",NaN
1,Hombre,29,Argentina,Corrientes,5.0,2.0,2.0,4,Universitario,En curso,...,Jamás,Heterosexual,Visual,No,201-500,Otras industrias,8,9,"Horarios flexibles, Stock options / RSUs, Viát...",NaN
2,Mujer,22,Argentina,Ciudad Autónoma de Buenos Aires,2.0,0.0,0.0,0,Secundario,Completado,...,En un trabajo anterior,Bisexual o queer,NaN,No,2001-5000,Otras industrias,6,9,"Clases de gimnasia online, Comidas pagas / sub...",NaN
3,Hombre,39,Argentina,Corrientes,15.0,15.0,13.0,5,Posgrado,Incompleto,...,En mi trabajo actual,Heterosexual,NaN,NaN,501-1000,Otras industrias,7,6,"Abono de celular, Capacitaciones y/o cursos, C...",NaN
4,Hombre,32,Argentina,Ciudad Autónoma de Buenos Aires,9.0,6.0,6.0,0,Universitario,En curso,...,Jamás,Heterosexual,NaN,NaN,11-50,Servicios / Consultoría de Software / Digital,6,6,"Clases de idiomas, Horarios flexibles, Vacacio...",NaN


In [13]:
df.columns

Index(['profile_gender', 'profile_age', 'work_country', 'work_province',
       'profile_years_experience', 'work_years_in_company',
       'work_years_in_current_position', 'work_people_in_charge_of',
       'profile_studies_level', 'profile_studies_level_state',
       'profile_career', 'profile_university',
       'profile_specialization_courses', 'profile_open_source_contributions',
       'profile_programming_as_hobby', 'work_role', 'tools_platform',
       'tools_programming_languages', 'tools_frameworks', 'tools_data_bases',
       'tools_qa_testing', 'tools_IDEs', 'tools_work_pc_os',
       'tools_cellphone_os', 'work_on_call_duty', 'salary_on_call_duty_charge',
       'work_on_call_duty_charge_type', 'work_contract_type',
       'salary_monthly_BRUTO', 'salary_monthly_NETO', 'salary_in_usd',
       'salary_satisfaction', 'salary_comparison_last_semester',
       'salary_has_bonus', 'salary_bonus_tied_to',
       'salary_inflation_adjustment_2020',
       'salary_percentage_inf

In [10]:
engine = create_engine('sqlite:///sysarmy.sqlite3', echo=True)

In [11]:
df.to_sql('survey', con=engine, if_exists="replace")

2022-04-21 12:51:33,197 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("survey")
2022-04-21 12:51:33,202 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 12:51:33,205 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("survey")
2022-04-21 12:51:33,207 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 12:51:33,221 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 12:51:33,227 INFO sqlalchemy.engine.Engine 
CREATE TABLE survey (
	"index" BIGINT, 
	profile_gender TEXT, 
	profile_age BIGINT, 
	work_country TEXT, 
	work_province TEXT, 
	profile_years_experience FLOAT, 
	work_years_in_company FLOAT, 
	work_years_in_current_position FLOAT, 
	work_people_in_charge_of BIGINT, 
	profile_studies_level TEXT, 
	profile_studies_level_state TEXT, 
	profile_career TEXT, 
	profile_university TEXT, 
	profile_specialization_courses TEXT, 
	profile_open_source_contributions TEXT, 
	profile_programming_as_hobby TEXT, 
	work_role TEXT, 
	tools_platform TEXT, 
	tools_programming_lang

In [51]:
query1 = "SELECT * FROM survey WHERE salary_monthly_NETO > 100000 LIMIT 10"
query2 = "SELECT COUNT(1) FROM survey WHERE salary_monthly_NETO > 100000"
query3 = "SELECT AVG(salary_monthly_NETO) FROM survey"
query4 = "SELECT AVG(salary_monthly_NETO) FROM survey WHERE profile_gender = 'Mujer'"
query5 = "SELECT AVG(salary_monthly_NETO) FROM survey WHERE profile_gender = 'Hombre'"
query6 = "SELECT profile_gender, AVG(salary_monthly_NETO) FROM survey GROUP BY profile_gender"
query7 = """
SELECT profile_gender, AVG(salary_monthly_NETO) 
FROM survey 
WHERE profile_years_experience > 5
GROUP BY profile_gender
"""
query8 = """
SELECT profile_gender, AVG(salary_monthly_NETO) as avg_salary
FROM survey 
WHERE profile_years_experience > 5
GROUP BY profile_gender
HAVING COUNT(*) > 100
ORDER BY  AVG(salary_monthly_NETO) DESC
"""
queries = [query1, query2, query3, query4, query5, query6, query7, query8]

In [52]:
with engine.connect() as con:
    for query in queries:
      rs = con.execute(query)
      print(query)
      for row in rs:
          print(row)

      print('\n\n')

2022-04-21 13:28:40,163 INFO sqlalchemy.engine.Engine SELECT * FROM survey WHERE salary_monthly_NETO > 100000 LIMIT 10
2022-04-21 13:28:40,165 INFO sqlalchemy.engine.Engine [raw sql] ()
SELECT * FROM survey WHERE salary_monthly_NETO > 100000 LIMIT 10
(2, 'Mujer', 22, 'Argentina', 'Ciudad Autónoma de Buenos Aires', 2.0, 0.0, 0.0, 0, 'Secundario', 'Completado', None, None, 'Sí, de forma particular, Sí, los pagó un empleador', 'No', 'Sí', 'Infosec', 'Linux, VMWare', 'Ninguno', 'Ninguno', 'Ninguna', 'Ninguna', 'Ninguno', 'Windows', 'iOS', 'No', 0.0, 'Porcentaje de mi sueldo', 'Full-Time', 160000.0, 127000.0, None, 3, 3, 'Un sueldo', 'Performance individual', 'No', 0.0, 0, 'En un trabajo anterior', 'Bisexual o queer', None, 'No', '2001-5000', 'Otras industrias', 6, 9, 'Clases de gimnasia online, Comidas pagas / subvencionadas, Descuentos varios (Mercado Pago, Clarín 365, Club La Nación, etc)', None)
(3, 'Hombre', 39, 'Argentina', 'Corrientes', 15.0, 15.0, 13.0, 5, 'Posgrado', 'Incompleto', 

In [17]:
conn = engine.connect()

In [20]:
result = conn.execute('SELECT COUNT(1) FROM survey WHERE salary_monthly_NETO > 100000')
print(result.all())

2022-04-21 13:02:34,361 INFO sqlalchemy.engine.Engine SELECT COUNT(1) FROM survey WHERE salary_monthly_NETO > 100000
2022-04-21 13:02:34,374 INFO sqlalchemy.engine.Engine [raw sql] ()
[(1657,)]
